In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr

In [2]:
df1 = pd.read_csv('23-07-2018-TO-23-07-2019MANALIPETCALLN.csv')
df2 = pd.read_csv('24-07-2019-TO-23-07-2020MANALIPETCALLN.csv')
df3 = pd.read_csv('24-07-2020-TO-24-07-2021MANALIPETCALLN.csv')
df4 = pd.read_csv('25-07-2021-TO-25-07-2022MANALIPETCALLN.csv')

df5 = pd.concat([df1,df2,df3,df4])

data = df5[['Open Price','High Price','Low Price','Average Price','Total Traded Quantity','Turnover','No. of Trades','Deliverable Qty','Close Price']]
data['Deliverable Qty']=data['Deliverable Qty'].replace('-',method='ffill')
data = np.array(data)
# plt.plot(data)

/var/folders/dy/dqwjhxdd5m9bxwxj_zz8j4v40000gn/T/ipykernel_24193/3870759969.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Deliverable Qty']=data['Deliverable Qty'].replace('-',method='ffill')


In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))
df = scaler.fit_transform(data)

In [12]:
target = df[:,-1]
from keras.preprocessing.sequence import TimeseriesGenerator
TimeseriesGenerator(df, target,
                               length=10, sampling_rate=2,
                               batch_size=2)

In [ ]:
def create_dataset(df, timestep):
    dataX,dataY = [],[]
    for i in range (len(df)-timestep-1):
        a = df[i:(i+timestep), 0]
        dataX.append(a)
        dataY.append(df[i+timestep,0])
    return np.array(dataX), np.array(dataY)

In [ ]:
## split the dataset into train and test split
training_size = int(len(df)*0.80)
test_size = len(df)-training_size
train_data, test_data = df[0:training_size,:], df[training_size:len(df),:1]

X_train, y_train = create_dataset(train_data, 100)
X_test, y_test = create_dataset(test_data, 100)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=128,verbose=1)

In [ ]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [ ]:
train_pred = scaler.inverse_transform(train_pred)
test_pred = scaler.inverse_transform(test_pred)

import math
from sklearn.metrics import mean_squared_error
display(math.sqrt(mean_squared_error(y_train, train_pred)),math.sqrt(mean_squared_error(y_test, test_pred)))

In [ ]:
look_back = 100
trainPredictPlot = np.empty_like(df)
trainPredictPlot[:,:]= np.nan
trainPredictPlot[look_back:len(train_pred)+look_back,:] = train_pred

testPredictPlot = np.empty_like(df)
testPredictPlot[:,:] = np.nan
testPredictPlot[len(train_pred)+(look_back*2)+1:len(df)-1,:] = test_pred

plt.figure(figsize=(20,10))
plt.plot(scaler.inverse_transform(df))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
m = X_test[-1].reshape(1,100,1)
data = []
for i in range(5):
    a = model.predict(m)
    print('price on day '+str(i)+': '+str(scaler.inverse_transform(a)))
    data.append(scaler.inverse_transform(a))
    m = np.insert(m,100,a)
    m = np.delete(m,0).reshape(1,100,1)

In [ ]:
test_pred

In [ ]:
df5

In [ ]:
X_test